In [1]:
lat_tls = 43.6016
lng_tls = 1.4407

In [2]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='3HKNONV5WQ0YZJUOEYSE0221FMCZQDZHKNQ0W4ZG1P5XW2KW',
client_secret='0TIKF1OFORWQTIPFVVLMGHHQST3LPPTBLMKXATPVIUPFJXUU',
v='20180323',
ll=str(lat_tls) + ',' + str(lng_tls),
query='loisirs',
limit=200,
locale='fr',
radius=1800
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
items = data['response']['groups'][0]['items']

In [3]:
from pandas import json_normalize

dataframe = json_normalize(items) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Théâtre du Capitole,Opéra,Place du Capitole,43.604130,1.444332,"[{'label': 'display', 'lat': 43.60412979544746...",406,31000,FR,Toulouse,Midi-Pyrénées,France,"[Place du Capitole, 31000 Toulouse, France]",NaN,4c84b599d92ea093f9785f72
1,Hôtel de Ville de Toulouse (Capitole),Hôtel de ville,Place du Capitole,43.604523,1.444179,"[{'label': 'display', 'lat': 43.60452250885116...",429,31000,FR,Toulouse,Midi-Pyrénées,France,"[Place du Capitole (Rue Lafayette), 31000 Toul...",Rue Lafayette,4c03b2459a7920a19230d079
2,Musée des Augustins,Musée d'art,21 Rue de Metz,43.601170,1.446397,"[{'label': 'display', 'lat': 43.6011697382502,...",461,31000,FR,Toulouse,Midi-Pyrénées,France,"[21 Rue de Metz, 31000 Toulouse, France]",NaN,4b96adcbf964a52077dd34e3
3,Parc de la Prairie des Filtres,Parc,Cours Dillon,43.595638,1.436977,"[{'label': 'display', 'lat': 43.59563769241053...",728,31300,FR,Toulouse,Midi-Pyrénées,France,"[Cours Dillon, 31300 Toulouse, France]",NaN,4bc1c05df8219c7456bbb310
4,Rio Loco,Salle de concert,NaN,43.596345,1.436907,"[{'label': 'display', 'lat': 43.59634529848775...",660,NaN,FR,Toulouse,Midi-Pyrénées,France,"[Toulouse, France]",NaN,4df7a497fa76511d0048d1bb


In [4]:
categorie_labels = {
    # Watch a show
    "spectacle": ["Opéra", "Théâtre", "Salle de spectacle", "Complexe des arts de la scène"],
    "cinema": ["Multiplexe", "Cinéma indépendant", "Cinéma"],
    "music": ["Salle de concert", "Auditorium"],

    # Visit a place
    "monument": ["Hôtel de ville", "Base militaire", "Église", "Site historique"],

    # Play
    "sport": ["Stade de basketball", "Stade de football", "Café des sports", "Salle de danse", "Stade"],
    "game": ["Salle d'arcade", "Parc d'attractions"],

    # Interact with culture
    "exposition": ["Musée d'art", "Musée des sciences", "Galerie d'art", "Musée d'histoire", "École"],

    # Enjoy green
    "parc": ["Parc", "Jardin"]
}

In [5]:
for i, cat_val in enumerate(categorie_labels.values()):
    filt = list(map(lambda cat_venu: cat_venu in cat_val,
                    dataframe_filtered['categories']))
    dataframe_filtered.loc[filt,'categories'] = list(categorie_labels.keys())[i]

print(dataframe_filtered.shape)
dataframe_filtered["categories"].unique()

(33, 15)


array(['spectacle', 'monument', 'exposition', 'parc', 'music', 'cinema',
       'sport', 'Complexe des arts de la\xa0scène'], dtype=object)

In [6]:
import folium

toulouse_map = folium.Map(location=[lat_tls, lng_tls], zoom_start=15, tiles="cartoDB positron")

for lat, lng, name, cat in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['categories']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='#205E40',
        fill=True,
        opacity=0,
        fill_color='#3AAB74',
        fill_opacity=0.7,
        parse_html=False).add_to(toulouse_map)

toulouse_map

In [7]:
import geopandas
import pandas as pd

url = "https://data.toulouse-metropole.fr/explore/dataset/comptages-pietons/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

tls_pedestrian = geopandas.read_file(url)

In [8]:
tls_pedestrian.head()

,moment_dans_la_journee,comptage,adresse,transposee,jours,annee,geometry
0,midi,893,devant la brasserie rue Bayard,semaine_midi_2006,semaine,2006.0,POINT (1.44588 43.60783)
1,midi,700,devant la brasserie rue Bayard,samedi_midi_2015,samedi,2015.0,POINT (1.44588 43.60783)
2,soir,854,devant la brasserie rue Bayard,semaine_soir_2013,semaine,2013.0,POINT (1.44588 43.60783)
3,soir,1178,devant la brasserie rue Bayard,samedi_soir_2013,samedi,2013.0,POINT (1.44588 43.60783)
4,soir,925,devant la brasserie rue Bayard,samedi_soir_2012,samedi,2012.0,POINT (1.44588 43.60783)


In [9]:
# Remove useless features
tls_pedestrian.drop(["transposee", "jours"], axis=1, inplace=True)

# Slice most recent data
filt = tls_pedestrian['annee'] >= 2015
tls_pedestrian = tls_pedestrian.loc[filt]

# Transform geometry feature type into a simple lat-lng list
tls_pedestrian['latitude'] = tls_pedestrian['geometry'].bounds['miny']
tls_pedestrian['longitude'] = tls_pedestrian['geometry'].bounds['minx']

# Change features types and names
tls_pedestrian['comptage'] = tls_pedestrian['comptage'].astype(int)
tls_pedestrian['annee'] = tls_pedestrian['annee'].astype(str)
tls_pedestrian['latitude'] = tls_pedestrian['latitude'].astype(str)
tls_pedestrian['longitude'] = tls_pedestrian['longitude'].astype(str)

tls_pedestrian.columns = ["day_time", "count", "address", "year", "geometry", "latitude", "longitude"]

# Format day_time feature labels
day_lab = ["journee", "midi", "matin"]
tls_pedestrian["day_time"] = list(map(lambda time: 'day' if time in day_lab else 'night',
                                      tls_pedestrian["day_time"]))

# Clean the index
tls_pedestrian.reset_index(drop=True, inplace=True)

# Overview
tls_pedestrian.head()

,day_time,count,address,year,geometry,latitude,longitude
0,day,700,devant la brasserie rue Bayard,2015.0,POINT (1.44588 43.60783),43.60782556389234,1.445876790590295
1,day,1058,devant la brasserie rue Bayard,2016.0,POINT (1.44588 43.60783),43.60782556389234,1.445876790590295
2,day,436,devant la brasserie rue Bayard,2018.0,POINT (1.44588 43.60783),43.60782556389234,1.445876790590295
3,day,232,devant la brasserie rue Bayard,2016.0,POINT (1.44588 43.60783),43.60782556389234,1.445876790590295
4,day,227,devant la brasserie rue Bayard,2015.0,POINT (1.44588 43.60783),43.60782556389234,1.445876790590295


In [10]:
tls_ped_grpby = tls_pedestrian.groupby(['address', 'year'], as_index=False).median()

# Remove null values
filt = tls_ped_grpby['count'] != 0
tls_ped_grpby = tls_ped_grpby[filt]

# Average values between years
tls_ped_grpby = tls_ped_grpby.groupby(['address'], as_index=False).median().round()

tls_ped_grpby = tls_ped_grpby.join(
    tls_pedestrian[['address','latitude','longitude']].drop_duplicates().set_index('address'),
    on="address")

print(tls_ped_grpby.shape)
tls_ped_grpby.head(7)

(81, 4)


,address,count,latitude,longitude
0,1 Rue Bayard,812.0,43.60824205958672,1.446576244291821
1,1 passage de l'hôtel de Ville,102.0,43.60456740686743,1.444621823413681
2,1 place Jeanne d'Arc,2430.0,43.60857631673823,1.44587074534944
3,1 rue d'Austerlitz,952.0,43.60544214882909,1.447397826384669
4,10 rue des Lois,698.0,43.60515870014418,1.44227143410786
5,10 rue des Tourneurs,904.0,43.59980685406695,1.44526647441732
6,11 Allées du Président Roosevelt,1060.0,43.60559474389562,1.448230373315519


In [11]:
import branca

# Hex color function to transform values from specified cmap, here Oranges
colorscale = branca.colormap.linear.Oranges_09.scale(tls_ped_grpby['count'].min(), tls_ped_grpby['count'].max())

# Initial layout map
toulouse_map = folium.Map(location=[lat_tls, lng_tls], zoom_start=15, tiles="cartoDB positron")

# Adding counted pedestrian spots with circles
for lat, lng, address, count in zip(tls_ped_grpby['latitude'], tls_ped_grpby['longitude'], tls_ped_grpby['address'], tls_ped_grpby['count']):
    label = '{} | Count : {}'.format(address, int(count))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=(count/200)+3,
        popup=label,
        color='#535760',
        opacity=0.5,
        fill=True,
        fill_color=colorscale(count),
        fill_opacity=0.7,
        parse_html=False).add_to(toulouse_map)

# Adding cultural venues
for lat, lng, name, cat in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['categories']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='#205E40',
        fill=True,
        opacity=0,
        fill_color='#3AAB74',
        fill_opacity=0.7,
        parse_html=False).add_to(toulouse_map)

# Show map
toulouse_map

In [12]:
distance = dataframe_filtered[['name', 'lat', 'lng']]

distance['coord'] = list(zip(distance['lat'], distance['lng']))

tempo = pd.DataFrame()
i = 0
for lat, lng in zip(tls_ped_grpby['latitude'], tls_ped_grpby['longitude']):
    tempo[i] = list(map(lambda coord: geopy.distance.geodesic(coord, (lat,lng)).meters,
                        distance['coord']))
    i += 1
    
tempo.columns = tls_ped_grpby['address']
tempo = tempo.applymap(int)

distance = pd.concat([distance, tempo], axis='columns')

distance.head()

<ipython-input-12-38f6d01b2acd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance['coord'] = list(zip(distance['lat'], distance['lng']))


NameError: name 'geopy' is not defined

In [ ]:
import ipywidgets

In [ ]:
int_range_slider = ipywidgets.IntSlider(value=5, min=0, max=500, step=5)
int_range_slider

In [ ]:
# Initial layout map
toulouse_map = folium.Map(location=[lat_tls, lng_tls], zoom_start=15, tiles="cartoDB positron")

# Adding cultural venues radius
for lat, lng, name, cat in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['categories']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=int_range_slider.value,
        color='#205E40',
        fill=True,
        opacity=0,
        fill_color='#4A5870',
        fill_opacity=0.5,
        parse_html=False).add_to(toulouse_map)

# Adding counted pedestrian spots with circles
for lat, lng, address, count in zip(tls_ped_grpby['latitude'], tls_ped_grpby['longitude'], tls_ped_grpby['address'], tls_ped_grpby['count']):
    label = '{} | Count : {}'.format(address, int(count))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=(count/200)+3,
        popup=label,
        color='#535760',
        opacity=0.7,
        fill=True,
        fill_color=colorscale(count),
        fill_opacity=0.9,
        parse_html=False).add_to(toulouse_map)

# Adding cultural venues
for lat, lng, name, cat in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['categories']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='#205E40',
        fill=True,
        opacity=0,
        fill_color='#3AAB74',
        fill_opacity=0.7,
        parse_html=False).add_to(toulouse_map)

# Show map
toulouse_map